# Locality Preserving Projections: scikit-learn implementation

[Locality preserving projection (LPP)](http://people.cs.uchicago.edu/~xiaofei/LPP.html) was introduced by Xiaofei He and Partha Niyogi at NIPS 2003. They provide a MATLAB implementation, but we'll be implementing it using scikit-learn, which provides some nice features that will do much of the work for us.

LPP is a manifold learning technique for dimensionality reduction, useful when you have high-dimensional input that you suspect can be described well in only a few dimensions and want to work with it in a space of much lower dimensionality. The classic example, and the one implemented in the NIPS paper, is projecting images of faces to 2D space for visualization.

The algorithm is based on spectral graph theory, and the main idea is to construct a graph in which like samples are connected with large weights, then project the samples to the low-dimensional space while maintaining the neighborhood structure. It is a linear transformation (there is a kernalized version, which allows for non-linear projection).

In [4]:
import numpy as np

np.random.seed(12345)
X = np.random.rand(100, 4)

## Step 1: Construct the Adjacency Graph

An adjacency graph simply describes which input samples are connected. Inputs that are similar should be connected (i.e. they are neighbors), and inputs that are dissimilar should not be connected. A simple way to represent the adjacency graph is an $m \times m$ matrix, where $m$ is the number of samples you have. If input $\mathbf{x}_i$ is connected to $\mathbf{x}_j$, the entry of the matrix at row $i$ and column $j$ is $1$, otherwise it is $0$. The matrix is then symmetric (if $\mathbf{x}_i$ is connected to $\mathbf{x}_j$, then $\mathbf{x}_j$ is connected to $\mathbf{x}_i$) and it is likely sparse (depending on how neighbors are determined).

He and Niyogi describe two methods of determining which samples are close to one another:

* $\epsilon$-neighborhoods: nodes $i$ and $j$ are connected if $||\mathbf{x}_i - \mathbf{x}_j||^2_2 < \epsilon$
* $k$ nearest neighbor: nodes $i$ and $j$ are connected if $i$ is one of $k$ nearest neighbors of $j$ or vice versa (neighbors determined by Euclidean distance)

The authors also note that adjacency can be determined using other (less principled) methods and that LPP will attempt to preserve those neighborhoods.

scikit-learn has a subpackage (`neighbors`) for creating graphs and such.

In [6]:
from sklearn import neighbors

# generate a k nearest neighbors adjacency graph
A = neighbors.kneighbors_graph(X, 4, include_self=True)

# generate an epsilon-neighborhood graph
A = neighbors.radius_neighbors_graph(X, 0.02, include_self=True)